Datenbank vorbereiten
 - Indexierung erstellen

In [2]:
import sqlite3

conn = sqlite3.connect('data/SOLKAT_DACH.gpkg')
cursor = conn.cursor()

index_name = "GWR_EGID_INDEX"

create_index_sql = "CREATE INDEX {0} ON SOLKAT_CH_DACH(GWR_EGID)".format(index_name)

cursor.execute(create_index_sql)

conn.commit()
conn.close()

OperationalError: index GWR_EGID_INDEX already exists

Daten einlesen aus Building Datenbank

In [17]:
import sqlite3
import numpy as np
import pandas as pd

conn = sqlite3.connect('data/data.sqlite')                                
cursor = conn.cursor()

# query building DB
sql = "SELECT EGID, GGDENAME, GDEKT, GKODE, GKODN from building where GDEKT = 'VS';"

df_building = pd.read_sql_query(sql, conn)

print(df_building)

print(df_building.head())

# Verbindung schließen
conn.close()

#df_building.to_csv("data/EGID.csv", index=False)

             EGID   GGDENAME GDEKT        GKODE        GKODN
0       101154903  Brig-Glis    VS  2636742.189  1127654.168
1       101154904  Brig-Glis    VS  2643288.259  1129898.127
2       101154910  Brig-Glis    VS  2640149.923  1128530.206
3       101154912  Brig-Glis    VS  2639467.475  1128273.863
4       101154914  Brig-Glis    VS  2641762.855  1128817.126
...           ...        ...   ...          ...          ...
171330     960827    Zermatt    VS  2624304.774  1097125.348
171331     960828    Zermatt    VS  2624262.409  1097060.144
171332     960829    Zermatt    VS   2623919.66   1096879.18
171333     960830    Zermatt    VS  2624162.904  1097196.838
171334     960832    Zermatt    VS  2624221.318  1097240.894

[171335 rows x 5 columns]
        EGID   GGDENAME GDEKT        GKODE        GKODN
0  101154903  Brig-Glis    VS  2636742.189  1127654.168
1  101154904  Brig-Glis    VS  2643288.259  1129898.127
2  101154910  Brig-Glis    VS  2640149.923  1128530.206
3  101154912  Bri

Daten einlesen aus Dach Datenbank

In [18]:
import sqlite3
import numpy as np
import pandas as pd

conn = sqlite3.connect('data/SOLKAT_DACH.gpkg')
cursor = conn.cursor()

df_dach = pd.DataFrame()

# Inhalt von Dataframe löschen
df_dach.drop(index=df_dach.index, inplace=True)

# Schleife über jede Gruppe von 500 EGIDs
for i in range(0, len(df_building), 500):
    # Aktuelle Gruppe von EGIDs auswählen
    egid_group = df_building["EGID"].iloc[i:i+500]
    
    # WHERE-Klausel für die Abfrage erstellen
    where_clause = "WHERE " + " OR ".join(["GWR_EGID='{}'".format(egid) for egid in egid_group])
    
    # SQL-Abfrage ausführen
    sql =  """
        SELECT 
            SUM(FLAECHE) AS Summe_Flaeche, 
            SUM(STROMERTRAG) AS Summe_Stromertrag, 
            SUM(FLAECHE_KOLLEKTOREN) AS Summe_Flaeche_Kollektoren, 
            GWR_EGID AS EGID
        FROM 
            SOLKAT_CH_DACH
        {}
        GROUP BY
            GWR_EGID;
        """.format(where_clause)
    
    result = pd.read_sql_query(sql, conn)

    # Ergebnis dem bestehenden DataFrame "df_dach" hinzufügen
    df_dach = df_dach.append(result, ignore_index=True)


# Verbindung schließen
conn.close()


# Spalte EGID muss für merge gleicher Datentyp wie df_building/EGID haben
df_dach["EGID"] = df_dach["EGID"].astype(str)

# Mergen und Zeilen behalten, die in beiden Dataframes gefunden wurden
df_merge = pd.merge(df_building, df_dach, how='inner', on='EGID')

df_merge.to_csv("data/DachInfo.csv", index=False)


C:\Users\flavi\AppData\Local\Temp\ipykernel_61404\2449605787.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dach = df_dach.append(result, ignore_index=True)
C:\Users\flavi\AppData\Local\Temp\ipykernel_61404\2449605787.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dach = df_dach.append(result, ignore_index=True)
C:\Users\flavi\AppData\Local\Temp\ipykernel_61404\2449605787.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dach = df_dach.append(result, ignore_index=True)
C:\Users\flavi\AppData\Local\Temp\ipykernel_61404\2449605787.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dach = df_dach.append(result,